In [33]:
import numpy as np
import pandas as pd
import collections

In [21]:
# Words that are answers
word_answers = np.loadtxt("wordle-answers-alphabetical.txt", comments="#", delimiter="\n", unpack=False, dtype=str)
# Words that are accepted, not including answers
word_allowed = np.loadtxt("wordle-allowed-guesses.txt", comments="#", delimiter="\n", unpack=False, dtype=str)

num_answers = len(word_answers)
num_allowed = len(word_allowed)

# Separate into letters
# E.g. letter[0] is the list of all first letters of the words
letter_answers = list(zip(*[list(_) for _ in word_answers]))
letter_allowed = list(zip(*[list(_) for _ in word_allowed]))



5

### Summary statistics

In [39]:
# Most common letters
first = letter_answers[0]

c = collections.Counter(first)

dict(c.most_common())

{'s': 366,
 'c': 198,
 'b': 173,
 't': 149,
 'p': 142,
 'a': 141,
 'f': 136,
 'g': 115,
 'd': 111,
 'm': 107,
 'r': 105,
 'l': 88,
 'w': 83,
 'e': 72,
 'h': 69,
 'v': 43,
 'o': 41,
 'n': 37,
 'i': 34,
 'u': 33,
 'q': 23,
 'j': 20,
 'k': 20,
 'y': 6,
 'z': 3}